In [1]:
import numpy as np
import pandas as pd
import datetime as datetime
# import dataload
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
tqdm.pandas()
pd.set_option('display.max_rows', 500)

In [3]:
cd /content/drive/MyDrive/2021/날씨공모전/code

/content/drive/MyDrive/2021/날씨공모전/code


# 0. 데이터 로드

## 0-1. 전체 데이터셋

In [7]:
final_data = pd.read_csv('../data/final_0620.csv', encoding='cp949')
final_data['date'] = final_data['date'].apply(lambda x : pd.to_datetime(x, format='%Y-%m-%d'))

## 0-2. 집단별 데이터

In [9]:
adf_corr = pd.read_csv('../data/adf_corr_group_0617.csv', encoding='cp949')

In [11]:
final_data = final_data.merge(adf_corr[['sm_cat', 'group']], on='sm_cat')

In [12]:
final_data.head()

,date,sex,big_cat,sm_cat,qty,ratio,오늘날씨,미세먼지,오늘 비,기상정보,태풍정보,cnt,avg_ta,max_ta,min_ta,rn_day,rn_hr1,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa,공휴일여부,주말여부,연_2019,월_2,월_3,월_4,월_5,월_6,월_7,월_8,월_9,월_10,월_11,월_12,분기_2,분기_3,분기_4,요일_1,요일_2,요일_3,요일_4,요일_5,요일_6,봄,여름,가을,겨울,age_20,age_30,age_40,age_50,group
0,2018-01-01,1,식품,가공란,37.0,0.0,10.69387,4.09137,0.00000,0.00000,0.04440,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,정상_low
1,2018-01-01,1,식품,가공란,16.0,0.0,12.42969,6.35294,0.00000,0.51282,0.42417,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,정상_low
2,2018-01-01,1,식품,가공란,9.0,0.0,13.02249,4.88037,0.00000,0.40650,0.00000,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,정상_low
3,2018-01-01,1,식품,가공란,3.0,0.0,11.69080,4.53350,1.21951,0.90909,0.00000,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,정상_low
4,2018-01-01,0,식품,가공란,13.0,0.0,11.23032,3.61296,0.38910,2.22222,0.04060,0.480964,0.885198,5.464146,-2.88074,0.0,0.0,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,정상_low


In [14]:
# sh_data : [정상, high corr]
# ush_data : [비정상, high corr]
# sl_data : [정상, low corr]
# usl_data : [비정상, low corr]

sh_data = final_data[final_data.group=='정상_high']
ush_data = final_data[final_data.group=='비정상_high']
sl_data = final_data[final_data.group=='정상_low']
usl_data = final_data[final_data.group=='비정상_low']

# 정상 데이터 => VAR
# 비정상 데이터 => 공적분 관계가 존재하면 VECM

sh = sh_data[['date', 'sm_cat', 'qty']].groupby(['date', 'sm_cat']).sum().reset_index()
sh = pd.pivot_table(sh, values='qty', index='date', columns='sm_cat').fillna(0)
ush = ush_data[['date', 'sm_cat', 'qty']].groupby(['date', 'sm_cat']).sum().reset_index()
ush = pd.pivot_table(ush, values='qty', index='date', columns='sm_cat').fillna(0)

# qty와 상관관계가 높은 데이터만 사용
sh_item = list(sh.columns) # sm_cat
ush_item = list(ush.columns) # sm_cat

# 정규화 안함
ush.head()

sm_cat,가열식 가습기,갈비/찜/바비큐용 돈육,감귤/한라봉/오렌지,감자,건대추,건어물 진미채,건포도,게장류,견과류 밤,고추/피망/파프리카,곡물차,과일류,국내산 돈육,굴 생물,기능성 아이케어 화장품,기능성 화장품 세트,기초 화장용 로션,기초 화장용 미스트,기초 화장용 크림,기타 주스류,김치류,난방용 열풍기,남성 로션,남성 선케어,남성 세트,남성 쉐이빙,느타리버섯,다이어트보조식,다이어트용 헬스보충식품,대게/킹크랩,데오드란트,둥굴레차,딸기/복분자/블루베리,딸기우유,라디에이터,루테인/눈 영양제,마늘/생강,메이크업 브러쉬,명태/동태,무김치,...,저/무지방우유,전기온수기,전기장판,전통차,조개,주꾸미,참외/메론/수박,초음파식 가습기,초코우유,카페트매트,칼슘/철분 영양제,캡슐/POD커피,컨벡터,코코아/핫초코,콜라겐/코큐텐 영양제,클렌징 로션,클렌징 티슈,탁상/USB 선풍기,탄산수,탄산음료,토마토,파/양파,파김치,포도/거봉/체리,프로바이오틱스,프로폴리스/로얄젤리,해초류,핸드워시/스크럽,핸드크림,허브차,헤어 브러쉬,헤어스타일링용 흑채,헤어스프레이,헤어에센스,호박,홍차,환자식,황토매트,휴대용 선풍기,히터
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01,7.0,22.0,883.0,206.0,23.0,16.0,5.0,127.0,129.0,62.0,72.0,57.0,25.0,90.0,364.0,9.0,500.0,128.0,3025.0,136.0,144.0,0.0,99.0,5.0,121.0,41.0,14.0,745.0,84.0,23.0,24.0,27.0,107.0,53.0,10.0,249.0,51.0,198.0,8.0,61.0,...,106.0,8.0,69.0,542.0,190.0,50.0,30.0,252.0,137.0,11.0,143.0,148.0,19.0,104.0,193.0,18.0,47.0,0.0,204.0,825.0,130.0,107.0,14.0,14.0,333.0,109.0,65.0,71.0,351.0,140.0,64.0,54.0,35.0,408.0,76.0,30.0,48.0,4.0,1.0,119.0
2018-01-02,23.0,7.0,1019.0,216.0,29.0,25.0,28.0,74.0,423.0,88.0,90.0,95.0,49.0,125.0,324.0,14.0,645.0,185.0,3581.0,180.0,182.0,0.0,114.0,6.0,126.0,62.0,30.0,842.0,115.0,32.0,47.0,59.0,154.0,80.0,31.0,324.0,99.0,212.0,15.0,88.0,...,129.0,14.0,86.0,388.0,192.0,59.0,42.0,399.0,150.0,15.0,253.0,278.0,18.0,176.0,182.0,38.0,56.0,4.0,181.0,696.0,179.0,125.0,7.0,27.0,520.0,168.0,88.0,137.0,413.0,283.0,87.0,38.0,77.0,445.0,105.0,56.0,48.0,6.0,4.0,287.0
2018-01-03,12.0,5.0,1007.0,234.0,29.0,23.0,1.0,78.0,129.0,92.0,77.0,109.0,39.0,171.0,415.0,10.0,596.0,207.0,2934.0,168.0,185.0,0.0,86.0,10.0,147.0,59.0,38.0,700.0,105.0,28.0,39.0,85.0,139.0,61.0,23.0,285.0,67.0,208.0,13.0,74.0,...,116.0,12.0,75.0,256.0,177.0,56.0,52.0,412.0,114.0,9.0,243.0,230.0,29.0,140.0,123.0,28.0,42.0,2.0,204.0,706.0,161.0,149.0,10.0,22.0,467.0,160.0,39.0,96.0,392.0,210.0,96.0,50.0,70.0,408.0,112.0,39.0,49.0,7.0,1.0,284.0
2018-01-04,15.0,13.0,878.0,199.0,24.0,23.0,7.0,72.0,179.0,109.0,78.0,77.0,35.0,177.0,657.0,21.0,535.0,174.0,2655.0,140.0,157.0,1.0,102.0,7.0,102.0,54.0,18.0,639.0,106.0,36.0,43.0,65.0,137.0,55.0,33.0,234.0,87.0,269.0,13.0,63.0,...,112.0,17.0,67.0,283.0,187.0,52.0,38.0,494.0,135.0,12.0,223.0,233.0,30.0,144.0,120.0,29.0,37.0,1.0,193.0,685.0,118.0,139.0,6.0,23.0,339.0,116.0,61.0,95.0,337.0,175.0,65.0,32.0,62.0,391.0,99.0,39.0,65.0,7.0,3.0,242.0
2018-01-05,18.0,9.0,1078.0,160.0,31.0,22.0,6.0,40.0,104.0,81.0,70.0,73.0,40.0,229.0,643.0,43.0,531.0,134.0,3063.0,134.0,152.0,1.0,88.0,1.0,143.0,51.0,30.0,557.0,71.0,22.0,30.0,56.0,96.0,47.0,23.0,207.0,70.0,197.0,11.0,78.0,...,76.0,14.0,64.0,223.0,156.0,59.0,26.0,320.0,146.0,15.0,170.0,146.0,19.0,114.0,170.0,28.0,49.0,3.0,178.0,682.0,132.0,111.0,8.0,19.0,641.0,112.0,240.0,120.0,296.0,336.0,109.0,48.0,45.0,357.0,91.0,44.0,46.0,7.0,3.0,242.0


## 0-3. `cnt` 데이터

In [15]:
var_list = ['date', 'cnt']
cnt = sh_data.groupby(['date']).mean().reset_index()[var_list]
cnt.index = cnt['date']
cnt = cnt.drop('date', axis=1)
cnt.head()

,cnt
date,
2018-01-01,0.480964
2018-01-02,1.168411
2018-01-03,1.809684
2018-01-04,1.058606
2018-01-05,1.302522


## 0-4. 날씨 데이터

In [36]:
var_list = ['date', 'avg_ta', 'max_ta', 'min_ta', 'avg_ws', 'PM10', 'PM25', 'hm_max', 'sum_ss_hr', 'max_pa']
weather = sh_data.groupby(['date']).mean().reset_index()[var_list]
weather.index = weather['date']
weather = weather.drop('date', axis=1)
weather.head()

,avg_ta,max_ta,min_ta,avg_ws,PM10,PM25,hm_max,sum_ss_hr,max_pa
date,,,,,,,,,
2018-01-01,0.885198,5.464146,-2.880740,2.157094,44.015611,21.477208,61.923313,52.276365,1020.592385
2018-01-02,1.208896,6.014385,-3.135167,1.903604,59.179487,33.836826,74.169781,37.996677,1021.524802
2018-01-03,-0.798604,3.236083,-3.619615,2.623562,32.757902,18.242391,59.310396,53.216885,1023.277500
2018-01-04,-1.075625,2.345708,-4.636646,1.842729,38.685406,23.045234,63.961750,33.707167,1021.539615
2018-01-05,0.584646,5.053948,-2.273062,2.280938,46.151746,30.785155,71.747365,40.403240,1014.525396


# 1. 모델링

In [16]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.api import VAR
from statsmodels.tsa.vector_ar.vecm import VECM

## 1-1. 정상성 확인

### 1-1-1. cnt raw

In [17]:
cnt.describe()

,cnt
count,730.000000
mean,3.753489
std,4.604229
min,0.480964
25%,2.217923
50%,2.867121
75%,3.523048
max,33.171142


In [19]:
def adf_test(data, sm_cat):
    item = data[sm_cat]
    result = adfuller(item.values)
    if result[1] <= 0.05:
        test_res = '정상' # 기각
    else:
        test_res = '비정상' # 기각하지 못함 
    return pd.DataFrame([{'sm_cat':sm_cat, 'ADF_stats':result[0], 'p-value':result[1], 'test_res':test_res}])

adf_c = adf_test(cnt, 'cnt') #cnt
adf_c

,sm_cat,ADF_stats,p-value,test_res
0,cnt,-4.147736,0.000807,정상


`cnt` : 정상 시계열

## 1-2. 정상 qty와 cnt(정상)

In [58]:
def VAR_granger(data, weather, sm_cat, w_var):
    x = pd.concat([weather[w_var], data[sm_cat]], axis=1)
    forecasting_model = VAR(x)
    results_aic = []
    for p in range(1,8):
        results = forecasting_model.fit(p)
        results_aic.append(results.aic)
    model = VAR(x)
    lag = np.argmin(results_aic) + 1
    var_res = model.fit(lag)
    g_res = var_res.test_causality(caused=sm_cat, causing=w_var)
    if g_res.pvalue <= 0.05:
        reject = 'causality'
    else:
        reject = 'None'
    return pd.DataFrame([{'sm_cat':sm_cat, 'weather':w_var, 'var_lag':lag, 'granger_reject':reject}])

In [59]:
warnings.filterwarnings('ignore')
sh_var_cnt = pd.DataFrame()

for sm_cat in sh_item:
    result = VAR_granger(sh, cnt, sm_cat, 'cnt')
    sh_var_cnt = pd.concat([sh_var_cnt, result], axis=0)

In [60]:
sh_var_cnt

,sm_cat,weather,var_lag,granger_reject
0,가스온수기,cnt,1,None
0,가자미,cnt,4,None
0,감/홍시,cnt,7,None
0,감말랭이,cnt,7,None
0,갓김치,cnt,7,causality
0,건강즙/녹용,cnt,4,None
0,건어물 멸치,cnt,7,None
0,건어물 황태,cnt,7,causality
0,건자두,cnt,7,None
0,견과류,cnt,2,None


In [61]:
sh_var_cnt.granger_reject.value_counts()

None         119
causality     11
Name: granger_reject, dtype: int64

In [51]:
# 예시
x = pd.concat([cnt['cnt'], sh['공기청정기']], axis=1)
model = VAR(x)
res = model.fit(8)
res.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sun, 20, Jun, 2021
Time:                     10:52:49
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    10.9275
Nobs:                     722.000    HQIC:                   10.7950
Log likelihood:          -5881.89    FPE:                    44880.7
AIC:                      10.7117    Det(Omega_mle):         42839.6
--------------------------------------------------------------------
Results for equation cnt
              coefficient       std. error           t-stat            prob
---------------------------------------------------------------------------
const            0.237009         0.085598            2.769           0.006
L1.cnt           0.896440         0.037536           23.882           0.000
L1.공기청정기         0.000604         0.000388            1.556           0.120
L2.cnt

## 1-3. 비정상 qty와 cnt(정상)

In [27]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen, select_coint_rank
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import grangercausalitytests

### 1-3-1. 공적분검정 & VECM

In [28]:
def co_test(data, weather, sm_cat, w_var):
    score, p_value, _ = coint(data[sm_cat], weather[w_var]) # coint(y, x)
    if p_value <= 0.05:
        test_res = '공적분' # 장기적으로 유의한 관계가 있다
    else:
        test_res = 'None' # 장기적으로 유의한 관계가 없다
    return pd.DataFrame([{'sm_cat':sm_cat, w_var:test_res}])

In [30]:
sh_co_cnt = pd.DataFrame()

w_sample = 'cnt'
res = pd.DataFrame()
for sm_cat in ush_item:
    res = pd.concat([res, co_test(ush, cnt, sm_cat, w_sample)], axis=0)

sh_co_cnt = pd.concat([sh_co_cnt, res], axis=1)

In [31]:
sh_co_cnt # 공적분 존재하는 품목이 갈비용 돈육뿐이라 그냥 다 VAR

,sm_cat,cnt
0,가열식 가습기,None
0,갈비/찜/바비큐용 돈육,공적분
0,감귤/한라봉/오렌지,None
0,감자,None
0,건대추,None
0,건어물 진미채,None
0,건포도,None
0,게장류,None
0,견과류 밤,None
0,고추/피망/파프리카,None


### 1-3-2. VAR

In [62]:
ush_var_cnt = pd.DataFrame()

for sm_cat in ush_item:
    ush_diff = ush.diff(1).fillna(0)
    if (adf_test(ush_diff, sm_cat).test_res=='정상').values:
        result = VAR_granger(ush_diff, cnt, sm_cat, 'cnt')
    else:
        result = pd.DataFrame([{'sm_cat':sm_cat, 'weather':'cnt', 'var_lag':lag, 'granger_reject':'None'}])
    ush_var_cnt = pd.concat([ush_var_cnt, result], axis=0)

In [63]:
ush_var_cnt

,sm_cat,weather,var_lag,granger_reject
0,가열식 가습기,cnt,7,None
0,갈비/찜/바비큐용 돈육,cnt,7,None
0,감귤/한라봉/오렌지,cnt,7,None
0,감자,cnt,7,None
0,건대추,cnt,6,None
0,건어물 진미채,cnt,7,None
0,건포도,cnt,7,None
0,게장류,cnt,7,None
0,견과류 밤,cnt,7,None
0,고추/피망/파프리카,cnt,7,None


In [64]:
ush_var_cnt.granger_reject.value_counts()

None         128
causality      6
Name: granger_reject, dtype: int64

In [65]:
ush_var_cnt[ush_var_cnt.granger_reject=='causality']

,sm_cat,weather,var_lag,granger_reject
0,물김치,cnt,7,causality
0,미나리,cnt,7,causality
0,애프터선,cnt,7,causality
0,야채/호박즙,cnt,7,causality
0,포도/거봉/체리,cnt,7,causality
0,허브차,cnt,7,causality


### cf. 날씨 코드 이상한거 발견해버림

In [33]:
st_weather = ['avg_ws', 'PM10', 'PM25', 'sum_ss_hr']
nst_weather = ['avg_ta', 'max_ta', 'min_ta', 'hm_max', 'max_pa']

In [43]:
ush_var_res = pd.DataFrame()

for w in st_weather:
    for sm_cat in ush_item:
        ush_diff = ush.diff(1).fillna(0)
        if (adf_test(ush_diff, sm_cat).test_res=='정상').values:
            result = VAR_granger(ush, weather, sm_cat, w)
        else:
            result = pd.DataFrame([{'sm_cat':sm_cat, 'weather':w, 'var_lag':lag, 'granger_reject':'None'}])
        ush_var_res = pd.concat([ush_var_res, result], axis=0)

In [46]:
for w in st_weather:
    print('===== Granger Causality : {} ====='.format(w))
    print(ush_var_res.loc[ush_var_res.weather==w, 'granger_reject'].value_counts())
    print(' ')

===== Granger Causality : avg_ws =====
None         118
causality     16
Name: granger_reject, dtype: int64
 
===== Granger Causality : PM10 =====
None         109
causality     25
Name: granger_reject, dtype: int64
 
===== Granger Causality : PM25 =====
None         122
causality     12
Name: granger_reject, dtype: int64
 
===== Granger Causality : sum_ss_hr =====
None         120
causality     14
Name: granger_reject, dtype: int64
 


In [52]:
ush_var_pivot = pd.pivot_table(ush_var_res, index='sm_cat', values='granger_reject', columns='weather', aggfunc='first')
ush_var_pivot

weather,PM10,PM25,avg_ws,sum_ss_hr
sm_cat,,,,
가열식 가습기,causality,causality,causality,None
갈비/찜/바비큐용 돈육,causality,None,None,None
감귤/한라봉/오렌지,None,None,None,None
감자,None,None,None,None
건대추,causality,None,causality,None
건어물 진미채,causality,None,None,None
건포도,None,None,None,None
게장류,None,None,None,None
견과류 밤,None,None,None,None


In [44]:
ush_var_res2 = pd.DataFrame()

for w in nst_weather:
    for sm_cat in ush_item:
        ush_diff = ush.diff(1).fillna(0); weather_diff = weather.diff(1).fillna(0)
        if ((adf_test(ush_diff, sm_cat).test_res=='정상').values)&((adf_test(weather_diff, w).test_res=='정상').values):
            result = VAR_granger(ush, weather_diff, sm_cat, w)
        else:
            result = pd.DataFrame([{'sm_cat':sm_cat, 'weather':w, 'var_lag':lag, 'granger_reject':'None'}])
        ush_var_res2 = pd.concat([ush_var_res2, result], axis=0)

In [45]:
for w in nst_weather:
    print('===== Granger Causality : {} ====='.format(w))
    print(ush_var_res2.loc[ush_var_res2.weather==w, 'granger_reject'].value_counts())
    print(' ')

===== Granger Causality : avg_ta =====
None         94
causality    40
Name: granger_reject, dtype: int64
 
===== Granger Causality : max_ta =====
None         103
causality     31
Name: granger_reject, dtype: int64
 
===== Granger Causality : min_ta =====
None         101
causality     33
Name: granger_reject, dtype: int64
 
===== Granger Causality : hm_max =====
None         100
causality     34
Name: granger_reject, dtype: int64
 
===== Granger Causality : max_pa =====
None         124
causality     10
Name: granger_reject, dtype: int64
 


In [53]:
ush_var_pivot2 = pd.pivot_table(ush_var_res2, index='sm_cat', values='granger_reject', columns='weather', aggfunc='first')
ush_var_pivot2

weather,avg_ta,hm_max,max_pa,max_ta,min_ta
sm_cat,,,,,
가열식 가습기,causality,causality,None,causality,causality
갈비/찜/바비큐용 돈육,None,None,None,None,None
감귤/한라봉/오렌지,causality,causality,None,None,causality
감자,None,None,None,None,None
건대추,None,None,None,None,causality
건어물 진미채,None,None,None,None,None
건포도,None,None,None,None,None
게장류,None,None,None,None,None
견과류 밤,None,None,None,None,None


In [54]:
ush_granger2 = pd.concat([ush_var_pivot, ush_var_pivot2], axis=1).reset_index()
ush_granger2['total'] = np.sum(ush_granger2.iloc[:,1:]=='causality', axis=1) # 인과관계가 있는 feature 개수

ush_granger2 

weather,sm_cat,PM10,PM25,avg_ws,sum_ss_hr,avg_ta,hm_max,max_pa,max_ta,min_ta,total
0,가열식 가습기,causality,causality,causality,None,causality,causality,None,causality,causality,7
1,갈비/찜/바비큐용 돈육,causality,None,None,None,None,None,None,None,None,1
2,감귤/한라봉/오렌지,None,None,None,None,causality,causality,None,None,causality,3
3,감자,None,None,None,None,None,None,None,None,None,0
4,건대추,causality,None,causality,None,None,None,None,None,causality,3
5,건어물 진미채,causality,None,None,None,None,None,None,None,None,1
6,건포도,None,None,None,None,None,None,None,None,None,0
7,게장류,None,None,None,None,None,None,None,None,None,0
8,견과류 밤,None,None,None,None,None,None,None,None,None,0
9,고추/피망/파프리카,None,None,None,None,None,None,None,None,None,0


In [56]:
print('인과관계가 하나라도 있는 상품의 개수 : {}'.format(len(ush_granger2[ush_granger2.total>0].sm_cat)))
print('인과관계가 하나도 없는 상품의 개수 : {}'.format(len(ush_granger2[ush_granger2.total==0].sm_cat)))

인과관계가 하나라도 있는 상품의 개수 : 79
인과관계가 하나도 없는 상품의 개수 : 55


In [57]:
ush_granger2[ush_granger2.total>0].sm_cat.unique() # 인과관계가 하나라도 있는 상품

array(['가열식 가습기', '갈비/찜/바비큐용 돈육', '감귤/한라봉/오렌지', '건대추', '건어물 진미채', '곡물차',
       '기능성 아이케어 화장품', '기능성 화장품 세트', '기초 화장용 로션', '기초 화장용 미스트',
       '기초 화장용 크림', '기타 주스류', '난방용 열풍기', '남성 선케어', '남성 세트', '남성 쉐이빙',
       '느타리버섯', '데오드란트', '둥굴레차', '딸기/복분자/블루베리', '라디에이터', '루테인/눈 영양제',
       '명태/동태', '미숫가루/곡물가루', '바디 보습제', '바디 클렌져', '바디케어용 땀패드', '바디케어용 제모제',
       '배추김치', '벽걸이 에어컨', '보리차', '복합식 가습기', '부추', '브로콜리/셀러리', '비타민',
       '색조 메이크업 립밤', '색조 메이크업 아이섀도우', '선스프레이', '선케어용 선밤', '소고기 육회', '쌈채소',
       '양배추/양상추', '어린이영양제', '에어워셔', '오메가3/스쿠알렌 영양제', '온수매트', '온열매트',
       '온풍기', '요거트/발효유', '유자차', '율무차', '자연식 가습기', '잡곡 씨드류', '장어', '전기온수기',
       '전기장판', '전통차', '초음파식 가습기', '초코우유', '카페트매트', '칼슘/철분 영양제',
       '캡슐/POD커피', '컨벡터', '코코아/핫초코', '콜라겐/코큐텐 영양제', '클렌징 로션',
       '탁상/USB 선풍기', '탄산수', '토마토', '파/양파', '파김치', '포도/거봉/체리',
       '프로폴리스/로얄젤리', '해초류 ', '핸드크림', '허브차', '황토매트', '휴대용 선풍기', '히터'],
      dtype=object)